In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [ ]:


# Set the seed for reproducibility
np.random.seed(101)

# Simulate data
n_samples = 10000

# Features
x1 = np.linspace(0, 10, n_samples)
x2 = np.random.uniform(-2, 2, n_samples)

# Additional features
x3 = np.random.normal(0, 1, n_samples)  # Normally distributed feature
x4 = np.random.beta(2, 5, n_samples)  # Feature with a beta distribution
x5 = np.random.choice([0, 1, 2, 3], size=n_samples)  # cat feature

# Updated target variable with more complex relationships
y = (
    3 * np.sin(x1) - 1.5 * np.cos(x2) +  # Sinusoidal and cosinusoidal relationships
    2 * np.square(x3) - x4**3 +  # Squared and cubic functions
    3 * x5 +  # Influence of the binary feature
    0.2 * x1 * x3 + 0.1 * x2 * x4 - 0.5 * x3 * x5 +  # Feature interactions
    np.random.normal(0, 1, n_samples)  # Noise
)

y = y/np.max(y)
# Update the DataFrame with the new features
data = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'x4': x4,
    'x5': x5,
    'y': y
})

# Display the first few rows of the updated DataFrame
data.head()


In [ ]:
from nampy.models import NAM

Initialize simple NAM with no interaction effects and MLPs for each feature.
You could also use a formuala such as:

"y ~ -1 + MLP(x1; Dropout=0.1, hidden_dims=[128, 128, 128]) + ResNet(x2) + RandomFourierNet(x3) + MLP(x4):MLP(x5)"

To define a NAM with an interaction effect and different shape functions for different features

In [ ]:

nam = NAM(
    "y ~ -1 + MLP(x1; Dropout=0.1) + MLP(x2) + MLP(x3) + MLP(x4) + MLP(x5)",   
    data=data, 
    feature_dropout=0.0001
    )

In [ ]:
nam.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss={"output":"mse"},
    metrics={"output":"mae"},
)


In [ ]:

# Train the model
nam.fit(nam.training_dataset, epochs=250, validation_data=nam.validation_dataset)

# Evaluate the model
loss = nam.evaluate(nam.validation_dataset)
print("Test Loss:", loss)

In [ ]:
nam.plot(port=8052)

In [ ]:
nam.plot(interactive=False, hist=False)